Notebook References:
- xarray access: https://github.com/EOPF-Sample-Service/eopf-sample-notebooks/blob/main/notebooks/Sentinel-2/Sentinel-2_L1C_MSI_Zarr_product_exploration.ipynb
- xcube-eopf: https://eopf-sample-service.github.io/eopf-sample-notebooks/introduction-xcube-eopf-plugin
- xcube-stac: https://github.com/xcube-dev/xcube-stac/blob/main/examples/notebooks/cdse_sentinel_2.ipynb

In [14]:
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
#import xarray_eopf

import dask
#import xcube
from xcube.core.store import new_data_store
from xcube_eopf.utils import reproject_bbox
#import xcube_eopf
#import xcube_stac

Items:
- eopf: https://stac.core.eopf.eodc.eu/collections/sentinel-2-l1c/items/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458
- cdse: https://stac.dataspace.copernicus.eu/v1/collections/sentinel-2-l1c/items/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458

```json
"file:size": 3734296,
      "proj:bbox": [300000, 4890240, 409800, 5000040],
      "proj:code": "EPSG:32632",
      "view:azimuth": 273.991839964449,
      "file:checksum": "1620f80c05b39730b386b0ebc9308af5120d8a0aafcb6a138cf2536dd4aff05ca5bc",
      "proj:transform": [60, 0, 300000, 0, -60, 5000040],
```


Browser:
- eopf: https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l1c/items/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458?.language=de
- cdse: https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-l1c/items/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458?.language=de

## 1. Read directly from link

In [2]:
# todo: is it possible to access a file directly via xcube?
path_eopf_zarr = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_v253/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458.zarr"
path_eopf_safe = "" # make available somehow on same bucket as zarr
path_cdse_safe = ""

## 2. Read via querry
The influence of the underlying STAC API (EOPF vs CDSE) and the implementation of the used software (xcube-stac vs xcube-eopf) affects the performance.

In [36]:
# todo: make dictionary for testing multiple combinations
time = ["2025-01-12", "2025-01-14"] # day
time = ["2025-01-01", "2025-02-01"] # month # ideally ["2024-01-01", "2024-02-01"]
time = ["2024-01-01", "2025-01-01"] # year

In [20]:
bbox = [6.45686051699677, 44.1379967481966, 7.87191870410008, 45.1480731828712] # stac bbox

# full scene
# quarter scene
# eight scene
# ml patch 256*256
# pixel

In [21]:
crs_utm = "EPSG:32632"

In [22]:
bbox_utm = reproject_bbox(bbox, "EPSG:4326", crs_utm)

In [23]:
bands = ["b02"]
# 1
# 2
# all one resolution
# all bands

In [24]:
spat_res = 10

In [25]:
store = new_data_store("eopf-zarr")

In [26]:
store.list_data_ids()

['sentinel-2-l1c', 'sentinel-2-l2a']

In [27]:
store.get_open_data_params_schema(data_id="sentinel-2-l1c")

In [47]:
%%time
ds = store.open_data(
    data_id="sentinel-2-l1c",
    bbox=bbox,
    time_range=time,
    spatial_res=spat_res,
    crs=crs_utm,
    variables=bands,
)
ds

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 479 ms


DataStoreError: No items found for search_params {'collections': ['sentinel-2-l2a'], 'datetime': ['2025-01-01', '2025-02-01'], 'bbox': (4.51134436157643, 0.00048253666454030244, 4.511346153376581, 0.00048343860342591805), 'query': None}.

In [52]:
# konstantins notebook: https://eopf-sample-service.github.io/eopf-sample-notebooks/introduction-xcube-eopf-plugin
time = ["2025-05-01", "2025-05-07"]
bbox = [9.85, 53.5, 10.05, 53.6]
crs_utm = "EPSG:32632"
bbox_utm = reproject_bbox(bbox, "EPSG:4326", crs_utm)

# returns:
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2A_MSIL2A_20250503T103701_N0511_R008_T32UNE_20250503T173316?.language=de
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2C_MSIL2A_20250501T104041_N0511_R008_T32UNE_20250501T161558?.language=de
# https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a/items/S2B_MSIL2A_20250506T103629_N0511_R008_T32UNE_20250506T115207?.language=de

In [54]:
%%time
ds = store.open_data(
    data_id="sentinel-2-l2a",
    bbox=bbox_utm,
    time_range=time,
    spatial_res=10,
    crs=crs_utm,
    variables=["b02", "b03", "b04", "scl"],
)
ds

CPU times: user 788 ms, sys: 19.2 ms, total: 807 ms
Wall time: 5.38 s


<xarray.Dataset> Size: 114MB
Dimensions:      (time: 3, y: 1132, x: 1341)
Coordinates:
  * time         (time) datetime64[ns] 24B 2025-05-01T10:40:41.025000 ... 202...
    spatial_ref  int64 8B ...
  * x            (x) float64 11kB 5.562e+05 5.563e+05 ... 5.696e+05 5.696e+05
  * y            (y) float64 9kB 5.94e+06 5.94e+06 ... 5.928e+06 5.928e+06
Data variables:
    b02          (time, y, x) float64 36MB dask.array<chunksize=(1, 1132, 1341), meta=np.ndarray>
    b03          (time, y, x) float64 36MB dask.array<chunksize=(1, 1132, 1341), meta=np.ndarray>
    b04          (time, y, x) float64 36MB dask.array<chunksize=(1, 1132, 1341), meta=np.ndarray>
    scl          (time, y, x) uint8 5MB dask.array<chunksize=(1, 1132, 1341), meta=np.ndarray>
Attributes: (4)

# 